## 1- Importing the libraries

In [15]:
import cv2
import numpy as np
import math

## 2- Make the desired functions

- Suppression Function
> This function filters a list of detected circles to remove redundant overlaps, keeping only the more “significant” ones and discarding others            that overlap too much.
- Is_Overlapping Function
> This function decides whether two circles should be considered “overlapping too much.” Its job is to compare two circles and return True if they       overlap, or False otherwise.

In [16]:
def suppression(circles):
    """
    Apply suppression to remove overlapping circles
    """
    if circles is None or len(circles) == 0:
        return []

    circles = [(x, y, r) for (x, y, r) in circles]
    suppressed = []
    
    while len(circles) > 0:
        current = circles.pop(0)
        suppressed.append(current)

        circles = [circle for circle in circles if not is_overlapping(current, circle)]
    
    return suppressed

def is_overlapping(circle1, circle2):
    """
    Check if two circles overlap
    """
    x1, y1, r1 = circle1
    x2, y2, r2 = circle2
    
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    if distance >= r1 + r2:
        return False
    elif distance <= abs(r1 - r2):
        return True
    else:
        return True

- get_coin_color_type Function
> This function analyzes the color characteristics of a detected coin and classifies it into one of three types: white, bronze, or gray. It works by isolating the circular coin region using a mask and then computing the average color of that region in both HSV and LAB color spaces.

In [17]:
def get_coin_color_type(img, x, y, r):
    """
    Determine coin color type: white, bronze, or gray
    """
    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    cv2.circle(mask, (x, y), r, 255, -1)
    
    roi = cv2.bitwise_and(img, img, mask=mask)
    
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mean_hsv = cv2.mean(hsv_roi, mask=mask)
    
    lab_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    mean_lab = cv2.mean(lab_roi, mask=mask)
    
    hue = mean_hsv[0]        
    saturation = mean_hsv[1] 
    value = mean_hsv[2]      
    lightness = mean_lab[0]  
    
    # Color classification rules:
    
    # White coins
    if value > 160 and saturation < 60:
        return "white"
    
    # Bronze coins
    elif 10 <= hue <= 30 and saturation > 40 and value > 80:
        return "bronze"
    
    # Gray coins
    elif saturation < 50 and 80 <= value <= 160:
        return "gray"
    
    # Default to bronze for coins that don't fit other categories
    else:
        return "bronze"

- classify_coin Fucntion
> This function assigns a coin denomination based on two factors: its radius and its detected color type (“white”, “bronze”, or “gray”). Using predefined size ranges for each color category, it determines which coin the measurement most likely corresponds to and returns both the coin’s label and its numeric value.

In [18]:
def classify_coin(radius, color_type):
    """
    Classify coin based on radius and color type according to the given rules
    """
    if color_type == "white":
        if radius < 100:
            return "5p", 5
        else:
            return "10p", 10
    elif color_type == "bronze":
        if radius > 90 and radius <= 115:
            return "1p", 1
        elif radius > 115 and radius <= 135:
            return "2p", 2
    elif color_type == "gray":
        if radius > 130:
            return "20p", 20

- detect_coins_with_value Function
> This function detects coins in an image, classifies them by color and size, calculates their individual denominations and total value, and optionally displays the annotated results. It combines a Hough Circle Transform and contour-based detection to improve accuracy, handles overlapping or missed coins, and provides detailed information about each coin, including its center, radius, color type, denomination, and value in pence.

In [19]:
def detect_coins_with_value(image_path, display_result=True):
    """
    Detects coins using Hough Circle and contour-based methods, classifies them by color and size, and calculates their total value.
    """
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None:
        print("Error: Could not load image")
        return 0, [], 0
    
    original = img.copy()
    height, width = img.shape[:2]
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    gray = cv2.GaussianBlur(gray, (9, 9), 2)
    gray = cv2.medianBlur(gray, 5)
    
    all_detections = []
    
    # Method 1: Hough Circles with optimized parameters
    circles = cv2.HoughCircles(gray, 
                              cv2.HOUGH_GRADIENT, 
                              dp=1.1,
                              minDist=40,
                              param1=45,
                              param2=22,
                              minRadius=70,
                              maxRadius=140)
    
    if circles is not None:
        hough_circles = np.round(circles[0, :]).astype("int")
        for circle in hough_circles:
            all_detections.append(tuple(circle))
        print(f"Hough Circles found: {len(hough_circles)}")
    
    # Method 2: Contour-based detection for missed coins
    contour_detections = 0
    for method in [cv2.THRESH_BINARY, cv2.THRESH_BINARY_INV, cv2.THRESH_OTSU]:
        if method == cv2.THRESH_OTSU:
            _, thresh = cv2.threshold(gray, 0, 255, method)
        else:
            _, thresh = cv2.threshold(gray, 100, 255, method)
        
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if 5000 < area < 50000: 
                (x, y), radius = cv2.minEnclosingCircle(cnt)
                radius = int(radius)
                
                # Check if this circle is similar to already detected ones
                if 70 <= radius <= 140:
                    all_detections.append((int(x), int(y), radius))
                    contour_detections += 1
    
    print(f"Contour detection found: {contour_detections} additional candidates")
    
    unique_detections = []
    for det in all_detections:
        if det not in unique_detections:
            unique_detections.append(det)
    
    print(f"Total unique detections before NMS: {len(unique_detections)}")
    
    final_circles = suppression(unique_detections)
    
    coin_count = 0
    radius_list = []
    coin_details = []
    total_value = 0
    
    for (x, y, r) in final_circles:
        radius = int(r)
        radius_list.append(radius)
        
        color_type = get_coin_color_type(original, x, y, radius)
        
        coin_type, coin_value = classify_coin(radius, color_type)
        
        coin_details.append({
            'center': (x, y),
            'radius': radius,
            'color_type': color_type,
            'type': coin_type,
            'value': coin_value
        })
        
        total_value += coin_value
        coin_count += 1
    
    coin_details.sort(key=lambda x: x['radius'])
    
    # Draw final detections with classification info
    for i, coin in enumerate(coin_details):
        x, y = coin['center']
        r = coin['radius']
        coin_type = coin['type']
        coin_value = coin['value']
        color_type = coin['color_type']
        
        if coin_type == "5p":
            color = (255, 255, 255)   
        elif coin_type == "10p":
            color = (255, 255, 255)   
        elif coin_type == "1p":
            color = (0, 100, 255)  
        elif coin_type == "2p":
            color = (0, 165, 255)   
        elif coin_type == "50p":
            color = (128, 128, 128) 
        else:
            color = (0, 255, 0)     
        

        cv2.circle(img, (x, y), r, color, 3)
        cv2.circle(img, (x, y), 2, (0, 0, 255), 5)
        

        cv2.putText(img, f"#{i+1}: {coin_type}", (x-40, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        

        cv2.putText(img, f"Color: {color_type}", (x-40, y+r+15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        
        print(f"Coin {i+1}: Center=({x}, {y}), Radius={r}, Color={color_type}, Type={coin_type}, Value={coin_value}p")
    

    cv2.putText(img, f"Total Coins: {coin_count}", (20, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    cv2.putText(img, f"Total Value: {total_value}p", (20, 60), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    
    print(f"\nHybrid method detected coins: {coin_count}")
    print("Coin details:")
    for i, coin in enumerate(coin_details, 1):
        print(f"  Coin {i}: {coin['type']} ({coin['value']}p), Radius={coin['radius']}, Color={coin['color_type']}")
    
    print(f"\nTotal value: {total_value}p")
    
    if display_result:
        cv2.imshow("Coin Detection with Classification", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return coin_count, radius_list, total_value

## 3- Display the results

In [ ]:
if __name__ == "__main__":
    image_path = 'image.png' 
    
    print("=== COIN DETECTION WITH VALUE CALCULATION ===")
    print("=" * 60)
    print("Classification Rules:")
    print("- White + radius < 100 = 5p")
    print("- White + radius ≥ 100 = 10p") 
    print("- Bronze + radius 90-110 = 1p")
    print("- Bronze + radius 110-130 = 2p")
    print("- Gray + radius > 130 = 50p")
    print("=" * 60)
    
    print("\n=== Hybrid Detection Method with Classification ===")
    coin_count, radii, total_value = detect_coins_with_value(image_path)
    
    print("\n" + "=" * 60)
    print("FINAL RESULTS:")
    print("=" * 60)
    print(f"Total coins detected: {coin_count}")
    print(f"Total value: {total_value}p")

=== COIN DETECTION WITH VALUE CALCULATION ===
Classification Rules:
- White + radius < 100 = 5p
- White + radius ≥ 100 = 10p
- Bronze + radius 90-110 = 1p
- Bronze + radius 110-130 = 2p
- Gray + radius > 130 = 50p

=== Hybrid Detection Method with Classification ===
Hough Circles found: 15
Contour detection found: 6 additional candidates
Total unique detections before NMS: 21
Coin 1: Center=(1632, 282), Radius=90, Color=white, Type=5p, Value=5p
Coin 2: Center=(994, 138), Radius=95, Color=bronze, Type=1p, Value=1p
Coin 3: Center=(1258, 466), Radius=96, Color=bronze, Type=1p, Value=1p
Coin 4: Center=(1206, 744), Radius=101, Color=bronze, Type=1p, Value=1p
Coin 5: Center=(976, 639), Radius=102, Color=bronze, Type=1p, Value=1p
Coin 6: Center=(1356, 255), Radius=124, Color=white, Type=10p, Value=10p
Coin 7: Center=(1418, 641), Radius=128, Color=bronze, Type=2p, Value=2p
Coin 8: Center=(1681, 509), Radius=135, Color=bronze, Type=2p, Value=2p
Coin 9: Center=(902, 369), Radius=139, Color=gray,